<a href="https://colab.research.google.com/github/JohnYechanJo/Novo-Nordisk_Anomaly-Detection/blob/main/Final_simple_runable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Synthetic CNV Image Classifier
This notebook uses CNV images from the OCT2017 dataset to generate synthetic CNV images using a Stable Diffusion model. The synthetic images are then mixed with Normal CNV images at various ratios (0% to 100%) to train a classifier. The goal is to find the optimal ratio of synthetic images to maximize classifier performance.

Execution Steps:

1. Data Preprocessing: OCT2017 CNV/NORMAL images → ViT embeddings → pre-trained_dataset.pt.

2. Diffusion Model Fine-tuning: Fine-tune Stable Diffusion UNet with CNV images.

3. Synthetic CNV Image Generation: Generate images using the fine-tuned model → ViT embeddings → synthetic_cnv_dataset.pt.

4. Classifier Training by Ratio: Mix data at synthetic ratios from 0% to 100% → Train classifier → Compare performance.

Execution Environment: Google Colab (GPU, e.g., T4 or A100 recommended).

##1. Environment Setup and Package Installation
Install the required Python packages and set the random seed to ensure reproducibility. GPU will be utilized, and functions for memory management will also be defined

In [3]:
!pip install kagglehub torch torchvision transformers diffusers accelerate datasets xformers pytorch-fid pandas
import os
import gc
import torch
import numpy as np
import random
from PIL import Image
import pandas as pd
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset, TensorDataset
from transformers import ViTModel, CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline
from accelerate import Accelerator
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from pytorch_fid import fid_score

# Set random seed
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
np.random.seed(seed)
random.seed(seed)

# GPU setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Memory cleanup function
def clear_memory():
    gc.collect()
    torch.cuda.empty_cache()

INFO: pip is looking at multiple versions of xformers to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB

##2. Diffusion Model Fine-tuning
Fine-tune the UNet of the Stable Diffusion model using CNV images.
Download the dataset from Kaggle, preprocess the CNV images, and use them for training.
The trained model will be saved to `/content/models/sd_cnv_finetuned.`

###2.1 Data Preparation
Download CNV images from the Kaggle dataset, resize them to 512x512, and save them to `/content/processed/CNV/.`

In [2]:
import kagglehub
import os
from PIL import Image

def prepare_cnv_images():
    # Kaggle dataset download
    path = kagglehub.dataset_download("paultimothymooney/kermany2018")
    print(f"Dataset downloaded to: {path}")

    # Set dataset path and copy CNV images
    in_dir = os.path.join(path, "OCT2017 /train/CNV")
    out_dir = "/content/processed/CNV/"
    os.makedirs(out_dir, exist_ok=True)
    for fn in os.listdir(in_dir):
        img = Image.open(os.path.join(in_dir, fn)).convert("RGB")
        img = img.resize((512, 512), resample=Image.LANCZOS)
        img.save(os.path.join(out_dir, fn))
    print(f"Processed CNV images saved to {out_dir}")

prepare_cnv_images()

Dataset downloaded to: /kaggle/input/kermany2018
Processed CNV images saved to /content/processed/CNV/


###2.2 Define Dataset Class
Define the `ImgDataset` class to load CNV images and transform them into the format required for training.

In [3]:
from torch.utils.data import Dataset
from torchvision import transforms

class ImgDataset(Dataset):
    def __init__(self, root_dir:str, tokenizer, cohort:str = "CNV", resolution=512, max_length=77, dataset_size:int = 6400) -> None:
        self.files = [
            os.path.join(root_dir, f)
            for f in os.listdir(root_dir)
            if f.lower().endswith((".png", ".jpg", ".jpeg"))
        ]
        if len(self.files) > dataset_size: self.files = self.files[:dataset_size]
        self.tokenizer = tokenizer
        self.transform = transforms.Compose([
            transforms.Resize((resolution, resolution), transforms.InterpolationMode.LANCZOS),
            transforms.ToTensor(),
            transforms.Normalize([0.5], [0.5]),
        ])
        self.prompt = f"OCT scan showing {cohort}"
        self.max_length = max_length

    def __len__(self) -> int:
        return len(self.files)

    def __getitem__(self, idx:int) -> dict:
        img = Image.open(self.files[idx]).convert("RGB")
        img = self.transform(img)
        tokens = self.tokenizer(
            self.prompt,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        return {"pixel_values": img, "input_ids": tokens.input_ids.squeeze(0)}

###2.3 Define Training Function
Define the `train` function to fine-tune the Stable Diffusion UNet using CNV images. It supports mixed precision training and checkpoint saving.

In [4]:
import torch
from torch.utils.data import DataLoader
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler
from transformers import CLIPTextModel, CLIPTokenizer
from accelerate import Accelerator

def train(
    pretrained_model: str,
    data_dir: str,
    output_dir: str = "sd_cnv_finetuned",
    cohort: str = "CNV",
    resolution: int = 512,
    dataset_size: int = 6400,
    batch_size: int = 4,
    learning_rate: float = 1e-4,
    epochs: int = 5,
    grad_accum_steps: int = 1,
    save_steps: int = 1000,
    resume_checkpoint: str = None,
    unet: UNet2DConditionModel = None,
    accelerator: Accelerator = None,
):
    # 1) Prepare output directory, accelerator & device
    os.makedirs(output_dir, exist_ok=True)
    accel = accelerator or Accelerator()
    device = accel.device

    # 2) Load or resume UNet
    if resume_checkpoint and unet is None:
        unet = UNet2DConditionModel.from_pretrained(resume_checkpoint).to(device)
    elif unet is None:
        unet = UNet2DConditionModel.from_pretrained(pretrained_model, subfolder="unet").to(device)

    # 3) Load & freeze tokenizer + text encoder
    tokenizer = CLIPTokenizer.from_pretrained(pretrained_model, subfolder="tokenizer")
    text_encoder = CLIPTextModel.from_pretrained(pretrained_model, subfolder="text_encoder").to(device)
    text_encoder.requires_grad_(False)

    # 4) Load & freeze VAE
    vae = AutoencoderKL.from_pretrained(pretrained_model, subfolder="vae").to(device)
    vae.requires_grad_(False)

    # 5) Load noise scheduler
    scheduler = DDPMScheduler.from_pretrained(pretrained_model, subfolder="scheduler")

    # 6) Prepare dataset & dataloader
    dataset = ImgDataset(data_dir, tokenizer, cohort=cohort, resolution=resolution, dataset_size=dataset_size)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    # 7) Optimizer (only UNet params)
    optimizer = torch.optim.AdamW(unet.parameters(), lr=learning_rate)

    # 8) Wrap models, optimizer, and dataloader for mixed-precision / distributed
    unet, optimizer, dataloader = accel.prepare(unet, optimizer, dataloader)

    # 9) Resume state if requested
    if resume_checkpoint:
        accel.load_state(resume_checkpoint)
        global_step = int(resume_checkpoint.rsplit("_", 1)[-1])
    else:
        global_step = 0

    # 10) Training loop
    for epoch in range(1, epochs + 1):
        unet.train()
        for batch in dataloader:
            with accel.accumulate(unet):
                # Encode images to latents
                pixels = batch["pixel_values"].to(device)
                latents = vae.encode(pixels).latent_dist.sample() * 0.18215

                # Add noise
                noise = torch.randn_like(latents)
                timesteps = torch.randint(0, scheduler.config.num_train_timesteps, (latents.shape[0],), device=device)
                noisy_latents = scheduler.add_noise(latents, noise, timesteps)

                # Text conditioning
                input_ids = batch["input_ids"].to(device)
                encoder_hidden_states = text_encoder(input_ids)[0]

                # Noise prediction & loss
                pred_noise = unet(noisy_latents, timesteps, encoder_hidden_states).sample
                loss = torch.nn.functional.mse_loss(pred_noise, noise)

                # Backpropagate
                accel.backward(loss)
                optimizer.step()
                optimizer.zero_grad()

            global_step += 1
            if global_step % save_steps == 0:
                accel.wait_for_everyone()
                ckpt_dir = os.path.join(output_dir, f"checkpoint_{global_step}")
                unet.save_pretrained(ckpt_dir)
                if accel.is_main_process:
                    tokenizer.save_pretrained(ckpt_dir)
                accel.save_state(ckpt_dir)

        print(f"Epoch {epoch}/{epochs} complete")

    # 11) Final save
    accel.wait_for_everyone()
    final_dir = os.path.join(output_dir, "final_unet")
    unet.save_pretrained(final_dir)
    if accel.is_main_process:
        tokenizer.save_pretrained(output_dir)
    print("Fine-tuning complete — models saved to", output_dir)

###2.4 Execute Diffusion Model Training
Use the pre-trained model (`nota-ai/bk-sdm-small`) to fine-tune with CNV images.

In [ ]:
from accelerate import Accelerator
# Memory clean up
clear_memory()
# Accelerator initialization
accel = Accelerator()

# Excute Diffusion Model training
train(
    pretrained_model="nota-ai/bk-sdm-small",
    data_dir="/content/processed/CNV",
    output_dir="/content/models/sd_cnv_finetuned",
    cohort="CNV",
    resolution=512,
    dataset_size=500,  # Consideration of Training Time
    batch_size=1,
    learning_rate=1e-4,
    epochs=5,
    grad_accum_steps=1,
    save_steps=1000,
    resume_checkpoint=None,
    accelerator=accel
)

# Memory clean up
clear_memory()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


config.json:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/1.93G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/806 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/492M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

scheduler_config.json:   0%|          | 0.00/313 [00:00<?, ?B/s]

##3. Generate Synthetic CNV Images
Use the fine-tuned Diffusion Model to generate synthetic CNV images, and create ViT embeddings. The embeddings will then be saved to `synthetic_cnv_dataset.pt.`

In [ ]:
# VIT Processing
def img_transform():
    return transforms.Compose([
        transforms.Lambda(lambda img: img.crop((0, 100, 768, 400))),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    #3，224，224

# Images Transfer to Tensors
def load_trans(path, pic_num=640):
    trans_toTensor = img_transform()
    image_list = []
    i = 0
    for filename in os.listdir(path):
        if i == pic_num:
            break
        file_path = os.path.join(path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                img = Image.open(file_path).convert("RGB")
                tensor_img = trans_toTensor(img)
                image_list.append(tensor_img)
            except Exception as e:
                print(f"Skip: {filename}, Error: {e}")
        i += 1
    return image_list

# Cell 10: Generate Synthetic CNV Images
def generate_synthetic_images():
    # Load the Base Pipeline from the Original Model
    pipeline = StableDiffusionPipeline.from_pretrained(
        "nota-ai/bk-sdm-small",
        torch_dtype=torch.float16,
        use_auth_token=False
    ).to(device)

    # Load the Fine-tuned UNet
    unet = UNet2DConditionModel.from_pretrained(
        "/content/models/sd_cnv_finetuned/final_unet",
        torch_dtype=torch.float16
    ).to(device)

    # Replace the UNet in the Pipeline
    pipeline.unet = unet

    # Generate Synthetic Images
    synthetic_dir = "/content/synthetic_cnv/"
    os.makedirs(synthetic_dir, exist_ok=True)
    num_images = 640
    prompt = (
     "OCT scan shows CNV"
    )


    for i in range(num_images):
        image = pipeline(prompt, num_inference_steps=50).images[0]
        image.save(os.path.join(synthetic_dir, f"synthetic_cnv_{i}.png"))
        if i % 50 == 0:
            print(f"Generated {i}/{num_images} images")
        clear_memory()

    # bsz, 3，224，224
    synthetic_tensor_list = load_trans(synthetic_dir, pic_num=num_images)


    # Save synthetic_cnv_dataset.pt
    if os.path.exists('synthetic_cnv_dataset.pt'):
        os.remove('synthetic_cnv_dataset.pt')
    torch.save(synthetic_tensor_list, 'synthetic_cnv_dataset.pt')
    print("Synthetic CNV dataset saved to synthetic_cnv_dataset.pt")

generate_synthetic_images()
clear_memory()

In [4]:
from google.colab import drive
drive.mount('/content/drive')
def img_transform():
    return transforms.Compose([
        transforms.Lambda(lambda img: img.crop((0, 100, 768, 400))),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    #3，224，224

# Images Transfer to Tensors
def load_trans(path, pic_num=640):
    trans_toTensor = img_transform()
    image_list = []
    i = 0
    for filename in os.listdir(path):
        if i == pic_num:
            break
        file_path = os.path.join(path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                img = Image.open(file_path).convert("RGB")
                tensor_img = trans_toTensor(img)
                image_list.append(tensor_img)
            except Exception as e:
                print(f"Skip: {filename}, Error: {e}")
        i += 1
    return image_list
synthetic_dir = "/content/drive/MyDrive/synthetic_cnv-2/synthetic_cnv"
synthetic_tensor_list = load_trans(synthetic_dir, pic_num=640)


# Save synthetic_cnv_dataset.pt
if os.path.exists('synthetic_cnv_dataset.pt'):
    os.remove('synthetic_cnv_dataset.pt')
torch.save(synthetic_tensor_list, 'synthetic_cnv_dataset.pt')
print("Synthetic CNV dataset saved to synthetic_cnv_dataset.pt")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Synthetic CNV dataset saved to synthetic_cnv_dataset.pt


In [12]:
import torch.nn as nn
def img_transform():
    return transforms.Compose([
        transforms.Lambda(lambda img: img.crop((0, 100, 768, 400))),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    #3，224，224

# Images Transfer to Tensors
def load_trans(path, pic_num=640):
    trans_toTensor = img_transform()
    image_list = []
    i = 0
    for filename in os.listdir(path):
        if i == pic_num:
            break
        file_path = os.path.join(path, filename)
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                img = Image.open(file_path).convert("RGB")
                tensor_img = trans_toTensor(img)
                image_list.append(tensor_img)
            except Exception as e:
                print(f"Skip: {filename}, Error: {e}")
        i += 1
    return image_list


class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv_block = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # (3,224,224) -> (16,224,224)
            nn.ReLU(),
            nn.MaxPool2d(2),                            # -> (16,112,112)

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # -> (32,112,112)
            nn.ReLU(),
            nn.MaxPool2d(2),                             # -> (32,56,56)

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # -> (64,56,56)
            nn.ReLU(),
            nn.MaxPool2d(2)                              # -> (64,28,28)
        )
        self.fc = nn.Sequential(
            nn.Flatten(),             # -> (64×28×28)
            nn.Linear(64*28*28, 128),
            nn.ReLU(),
            nn.Linear(128, 2)         # 2-class classification
        )

    def forward(self, x):
        x = self.conv_block(x)
        x = self.fc(x)
        return x
class Classifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = SimpleCNN().cuda()
        self.best_acc = 0
    def train_val_test(self):
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3)
        # unpack
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        if os.path.exists('Dataset.pt'):
         data_dic = torch.load('Dataset.pt')
         mixed_train = data_dic['train_data'].to(device)
         mixed_train_label = data_dic['train_label'].to(device)
         mixed_val = data_dic['val_data'].to(device)
         mixed_val_label = data_dic['val_label'].to(device)
         mixed_test = data_dic['test_data'].to(device)
         mixed_test_label = data_dic['test_label'].to(device)

        train_loader = DataLoader(TensorDataset(mixed_train, mixed_train_label), batch_size=128, shuffle=True)
        loss = nn.CrossEntropyLoss()
        epochs = 8
        for epoch in range(epochs):
            print(f"\nEpoch {epoch + 1}/{epochs}")
            # train
            self.model.train()
            for i, data in enumerate(train_loader):
                total = len(train_loader)
                batch_x, batch_y = (item.cuda() for item in data)
                self.optimizer.zero_grad()
                logit_original = self.model(batch_x)
                l = loss(logit_original, batch_y)
                l.backward()
                self.optimizer.step()
                corrects = (torch.max(logit_original, 1)[1].view(batch_y.size()).data == batch_y.data).sum()
                accuracy = 100 * corrects / len(batch_y)
                print(f'Batch[{i + 1}/{total}] - loss: {l.item():.6f}  accuracy: {accuracy:.4f}%({corrects}/{batch_y.size(0)})')
            # val
            self.model.eval()
            with torch.no_grad():
                logits = self.model(mixed_val)
                predicted = torch.max(logits,dim=1)[1]
                y_pred = predicted.data.cpu().numpy().tolist()
                acc = accuracy_score(mixed_val_label.cpu().numpy().tolist(), y_pred)
                print(f"Validation Accuracy: {acc:.4f}")
                if acc > self.best_acc:
                   self.best_acc = acc
                print("Best val set acc:", self.best_acc)
        # test
        self.model.eval()
        with torch.no_grad():
            logits = self.model(mixed_test)
            predicted = torch.max(logits,dim=1)[1]
            y_pred = predicted.data.cpu().numpy().tolist()
            try:
              res = classification_report(mixed_test_label.cpu().numpy().tolist(), y_pred, labels=[0, 1], target_names=['NR', 'FR'], digits=3, output_dict=True)
              for k, v in res.items():
                  print(k, v)
              print(f"result: {res['accuracy']:.4f}")
            except ValueError as e:
              print(f"Error in classification_report: {e}")
              res = {'accuracy': 0, 'macro avg': {'f1-score': 0, 'precision': 0, 'recall': 0}}
        return res




def runable(model):
    nn = model
    res = nn.train_val_test()
    return res


In [ ]:
import torch
import numpy as np
import pandas as pd
import os
import kagglehub

def train_classifier_with_ratios():
    results = []
    synthetic_images = torch.load('synthetic_cnv_dataset.pt')
    synthetic_images = torch.stack(synthetic_images)
    # download dataset
    path = kagglehub.dataset_download("paultimothymooney/kermany2018")
    loadpath = os.path.join(path, "OCT2017 /train")
    train_path_cnv = os.path.join(loadpath, "CNV")
    train_path_normal = os.path.join(loadpath, "NORMAL")
    # bsz,3,224,224
    cnv_tensor_list = load_trans(train_path_cnv, pic_num = 800)
    normal_tensor_list = load_trans(train_path_normal,  pic_num = 800)
    cnv_tensor_list = torch.stack(cnv_tensor_list)
    normal_tensor_list = torch.stack(normal_tensor_list)
    # ratio : 0~1
    ratios = [i/10 for i in range(11)]
    # batch_size = 128
    half_batch = 64
    batch_num = 10
    for ratio in ratios:
        mixed_train = []
        mixed_train_label = []
        num_synthetic_batch = int(64 * ratio)
        num_normal_cnv_batch = 64 - num_synthetic_batch

        # get train dataset
        for i in range(batch_num):
          labels = torch.cat([torch.ones(half_batch, dtype=torch.long),
                              torch.zeros(half_batch, dtype=torch.long)], dim=0)
          s_cnv = synthetic_images[i*num_synthetic_batch:(i+1)*num_synthetic_batch]
          n_cnv = cnv_tensor_list[i*num_normal_cnv_batch:(i+1)*num_normal_cnv_batch]
          n_norm = normal_tensor_list[i*half_batch:(i+1)*half_batch]
          assert all(isinstance(t, torch.Tensor) for t in mixed_train)
          batch = torch.cat([s_cnv, n_cnv, n_norm],dim=0)
          mixed_train.append(batch)
          mixed_train_label.append(labels)
        mixed_train = torch.cat(mixed_train, dim=0)
        mixed_train_label = torch.cat(mixed_train_label, dim=0)

        #get val dataset
        i, j = batch_num*num_normal_cnv_batch, batch_num*half_batch
        val_labels = torch.cat([torch.ones(80, dtype=torch.long),
                              torch.zeros(80, dtype=torch.long)], dim=0)
        n_cnv = cnv_tensor_list[i:i+80]
        n_norm = normal_tensor_list[j:j+80]
        mixed_val = torch.cat([n_cnv, n_norm],dim=0)

        #get test dataset
        i, j=i+80, j+80
        test_labels = torch.cat([torch.ones(80, dtype=torch.long),
                              torch.zeros(80, dtype=torch.long)], dim=0)
        n_cnv = cnv_tensor_list[i:i+80]
        n_norm = normal_tensor_list[j:j+80]
        mixed_test = torch.cat([n_cnv, n_norm],dim=0)

        # save norm->0 / cnv->1
        if os.path.exists('Dataset.pt'):
           os.remove('Dataset.pt')
        torch.save({
            "train_data": mixed_train,
            "train_label": mixed_train_label,
            "val_data": mixed_val,
            "val_label": val_labels,
            "test_data": mixed_test,
            "test_label": test_labels
        },"Dataset.pt")
        clear_memory()

        # Train and evaluate model
        print(f"Training model with ratio {ratio}")
        model = Classifier()  # Assumes Classifier is defined elsewhere
        res = runable(model)  # Assumes train_and_test is defined
        results.append({
            'ratio': ratio,
            'accuracy': res['accuracy'],
            'f1_score': res['macro avg']['f1-score'],
            'precision': res['macro avg']['precision'],
            'recall': res['macro avg']['recall']
        })
        clear_memory()  # Assumes clear_memory is defined elsewhere


    print("\nResults for Ratio 0 to 1:")
    for res in results:
        if res['accuracy'] is not None:
            print(f"Ratio: {res['ratio']*100:.0f}% | Accuracy: {res['accuracy']:.4f} | F1 Score: {res['f1_score']:.4f} | Precision: {res['precision']:.4f} | Recall: {res['recall']:.4f}")


    pd.DataFrame(results).to_csv('classifier_results.csv', index=False)
    print("Results saved to classifier_results.csv")

train_classifier_with_ratios()

Training model with ratio 0.0

Epoch 1/8
Batch[1/10] - loss: 0.692658  accuracy: 52.3438%(67/128)
Batch[2/10] - loss: 9.828728  accuracy: 47.6562%(61/128)
Batch[3/10] - loss: 1.260177  accuracy: 48.4375%(62/128)
Batch[4/10] - loss: 0.970105  accuracy: 53.1250%(68/128)
Batch[5/10] - loss: 0.795989  accuracy: 53.1250%(68/128)
Batch[6/10] - loss: 0.753871  accuracy: 49.2188%(63/128)
Batch[7/10] - loss: 0.705475  accuracy: 50.7812%(65/128)
Batch[8/10] - loss: 0.695393  accuracy: 51.5625%(66/128)
Batch[9/10] - loss: 0.693497  accuracy: 50.7812%(65/128)
Batch[10/10] - loss: 0.697413  accuracy: 42.9688%(55/128)
Validation Accuracy: 0.5125
Best val set acc: 0.5125

Epoch 2/8
Batch[1/10] - loss: 0.692842  accuracy: 47.6562%(61/128)
Batch[2/10] - loss: 0.695038  accuracy: 43.7500%(56/128)
Batch[3/10] - loss: 0.691345  accuracy: 53.9062%(69/128)
Batch[4/10] - loss: 0.689631  accuracy: 55.4688%(71/128)
Batch[5/10] - loss: 0.701452  accuracy: 46.0938%(59/128)
Batch[6/10] - loss: 0.698462  accuracy: